<a href="https://colab.research.google.com/github/andyarnell/Global_ecological_zones_mapping/blob/main/gez_comparison_sankey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aim: create Sankey plots to illustrate magnitude of climate zone changes to help compare GEZ update options

In [1]:
# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

install_if_not_exists("sankee")

import sankee

import ee # google earth engine
import geemap
from google.colab import output
output.enable_custom_widget_manager()

gee_project_name = "ee-andyarnellgee"

ee.Authenticate()

ee.Initialize(project=gee_project_name)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.0 MB/s eta 0:00:00
sankee has been installed.


In [3]:
chelsa_climate_1981_2010 = ee.ImageCollection("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/chelsa_tas_1981_2010");
chelsa_pet = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/chelsa_pet_penman_mean_1981-2010_V_2_1");
worldclim_climate = ee.ImageCollection("WORLDCLIM/V1/MONTHLY");
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE");
mountain_belts = ee.Image("users/xavidelamo/SDG1542_Mntn_BioclimaticBelts");
hzl_contemp = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/life_zones_contemporary");
# admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0");
# table = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");

In [15]:
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Function to calculate number of months above a temperature threshold
def calc_number_of_months_above(monthly_image_col, band_name_tavg, temp_threshold):
    def map_threshold(image):
        image_tavg = image.select(band_name_tavg)
        image_gte_thresh = image_tavg.gte(temp_threshold)
        return image_gte_thresh

    monthly_image_col_tavg = monthly_image_col.select(band_name_tavg)
    monthly_image_col_tavg_gte_thresh = monthly_image_col_tavg.map(map_threshold)
    count_of_months_above = monthly_image_col_tavg_gte_thresh.sum()
    return count_of_months_above

# Function to classify Köppen-Trewartha climate zones
def classify_koppen_trewartha(monthly_image_col, band_name_tavg, prec_avg, pet_avg, high_thresh, low_thresh):
    # Calculate number of months above 18 degrees and 10 degrees
    number_of_months_above_18_degrees = calc_number_of_months_above(monthly_image_col, band_name_tavg, high_thresh)
    number_of_months_above_10_degrees = calc_number_of_months_above(monthly_image_col, band_name_tavg, low_thresh)

    # Define Köppen-Trewartha rules
    threshold_18_deg = 12  # standard kt val :12 (but for toggling to get similar to GEZ level 1, 8 is closer!)
    high_threshold_10_deg = 8  # standard kt val :8
    middle_threshold_10_deg = 4  # standard kt val :4
    low_threshold_10_deg = 1  # standard kt val : 1

    # Define Köppen-Trewartha rules
    tropical = number_of_months_above_18_degrees.gte(threshold_18_deg)
    subtropical = number_of_months_above_10_degrees.gte(high_threshold_10_deg).And(number_of_months_above_18_degrees.lt(threshold_18_deg))
    temperate = number_of_months_above_10_degrees.gte(middle_threshold_10_deg).And(number_of_months_above_10_degrees.lt(high_threshold_10_deg))
    boreal = number_of_months_above_10_degrees.gte(low_threshold_10_deg).And(number_of_months_above_10_degrees.lt(middle_threshold_10_deg))
    polar = number_of_months_above_10_degrees.lt(low_threshold_10_deg)

    # Combine rules to create zones
    koppenTrewarthaZones = ee.Image(0) \
        .where(tropical, 1) \
        .where(subtropical, 2) \
        .where(temperate, 3) \
        .where(boreal, 4) \
        .where(polar, 5) \
        .selfMask()

    # Return the image with Köppen-Trewartha zones
    return koppenTrewarthaZones.rename('koppen_trewartha')



# Data prep

# Potential evapotranspiration
terraclimate_pet_avg = terraclimate.select('pet').filter(ee.Filter.date('1960-01-01', '1991-01-01')).mean()

# Precipitation
worldclim_climate_prec_avg = worldclim_climate.select('prec').mean()

# Mountains
mountains = mountain_belts.gt(1)

# Analysis
# Apply classify_koppen_trewartha function to worldclim
worldclim_koppen_trewartha = classify_koppen_trewartha(worldclim_climate,
                                                      'tavg',
                                                      worldclim_climate_prec_avg,
                                                      terraclimate_pet_avg,  # as not included in worldclim
                                                      180,
                                                      100)

# Apply classify_koppen_trewartha function to chelsa
chelsa_koppen_trewartha = classify_koppen_trewartha(chelsa_climate_1981_2010,
                                                      'b1',
                                                      worldclim_climate_prec_avg,  # using for temp interim
                                                      chelsa_pet,
                                                      18,
                                                      10)

# Mask to terrestrial areas (based on worldclim extent for now)
chelsa_koppen_trewartha = chelsa_koppen_trewartha.updateMask(worldclim_koppen_trewartha.gt(0))

# Make a version with mountains included (based on an asset used for reporting mountain green cover index)
chelsa_koppen_trewartha_w_mountains = chelsa_koppen_trewartha.where(mountains, 6)
worldclim_koppen_trewartha_w_mountains = worldclim_koppen_trewartha.where(mountains, 6)


In [113]:
# Load the existing GEZ data showing level 2
gez_2010_level2 = ee.Image('users/bornToBeAlive/gez_2010_wgs84').selfMask()

# Load the existing GEZ data showing level 1 (approximate climate zones)
gez_2010_level1 = gez_2010_level2.divide(10).floor().int().selfMask()

# Load the Holdridge Life Zones (adapted from Elsen et al 2022)
hzl_contemp = ee.Image("Your_Holdridge_Life_Zones_Image")  # Replace "Your_Holdridge_Life_Zones_Image" with the actual image asset ID

# Create a simplified version of the Holdridge Life Zones to approximate GEZ level 1
hzl_contemp_simple = hzl_contemp.divide(10).floor().int().selfMask()

# Create a map
Map = geemap.Map()

# Add layers to the map
# Map.addLayer(gez_2010_level2.randomVisualizer(), {}, "gez_2010_level2")
Map.addLayer(gez_2010_level1.randomVisualizer(), {}, "gez_2010_level1")

# Add data-driven zones based on climate datasets
Map.addLayer(worldclim_koppen_trewartha.select('koppen_trewartha').randomVisualizer(),{},'Worldclim Köppen-Trewartha Climate Zones');

Map.addLayer(chelsa_koppen_trewartha.select('koppen_trewartha').randomVisualizer(),{},'Chelsa Köppen-Trewartha Climate Zones');

# Map.addLayer(hzl_contemp, {}, "life zones contemporary - all")
# Map.addLayer(hzl_contemp_simple, {}, "life zones contemporary - simple")


# # Load two images
# pre = ee.Image("GOOGLE/DYNAMICWORLD/V1/20200904T185921_20200904T190750_T10TEQ")
# post = ee.Image("GOOGLE/DYNAMICWORLD/V1/20201009T190319_20201009T190349_T10TEQ")
aoi = ee.Geometry.Point([-122.30239918572622, 44.802882471354316]).buffer(5000000)

xMin=-180
yMin=-90
xMax=180
yMax=90

# aoi = ee.Geometry.Polygon([xMin, yMin, xMax, yMax], None,False)
aoi = ee.Geometry.Polygon([
    [-180, 90], [0, 90], [180, 90],
    [180,-90], [0, -90], [-180,-90]],
    None, False);

print (aoi.getInfo())
# print(pre.bandNames().getInfo())
Map.addLayer(aoi)
# Display the map
Map


{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-180, 90], [0, 90], [180, 90], [180, -90], [0, -90], [-180, -90], [-180, 90]]]}


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [121]:
pre = gez_2010_level1
post = worldclim_koppen_trewartha_w_mountains.select(0).rename("b1")


In [122]:
# Define the band name and the class labels and colors corresponding to each pixel value.
band = "b1"

labels = {
    1: "tropical",
    2: "subtropical",
    3: "temperate",
    4: "boreal",
    5: "polar",
}
palette = {
    # 0: "#419BDF",
    1: "#397D49",
    2: "#88B053",
    3: "#7A87C6",
    4: "#E49635",
    5: "#DFC35A",
    # 6: "#C4281B",
    # 7: "#A59B8F",
    # 8: "#B39FE1"
}


In [132]:

# Generate the Sankey diagram from the two images
sankee.sankify(
    image_list=[pre, post],
    region=aoi,
    band=band,
    labels=labels,
    palette=palette,
    scale=100,
    seed=11
)
# sankee.sankify(
#     img_list,
#     region=fire,
#     band=band,
#     labels=labels,
#     palette=palette,
#     label_list=label_list,
#     scale=30
# )

SankeyPlot()